<div>
    <p style="float: right;"><img width="66%" src="templates/logo_fmriflows.gif"></p>
    <h1>Multivariate Analysis</h1>
    <p>This notebook performes a simple multivariate analysis by executing the following steps:

1. Collect files and labels
1. Create mask to restrict searchlight analysis
1. Create PyMVPA dataset and z-score it
1. Select target samples
1. Run searchlight analysis
1. Create outputs 1st-level
1. Perform 2nd-level analysis (classical GLM and/or multivariate according to [Stelzer et al. (2013)](https://www.sciencedirect.com/science/article/pii/S1053811912009810))

**Note:** This notebook requires that the 1st-level analysis pipeline was already executed, with the parameter `con_per_run` set to `True`, and that it's output can be found in the dataset folder under `/dataset/derivatives/fmriflows/analysis_1stLevel/multivariate`. </p>
</div>

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── fmriflows_spec_multivariate.json
    └── derivatives
        └── fmriflows
            └── analysis_1stLevel
                └── multivariate
                    └── sub-{sub_id}
                        └── task-{task_id}
                            └── tFilter-{tFilter_id}_sFilter-{sFilter_id}
                                ├── con_[con_id]_norm.nii.gz
                                └── labels.csv

`fmriflows` will perform a multivariate (searchlight) analysis on each subjects individually. Type of classifier and possible binary classifications need to be specified in the `fmriflows_spec_multivariate.json` file.

## Execution Specifications

This notebook will extract the relevant analysis specifications from the `fmriflows_spec_multivariate.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'fmriflows_spec_multivariate.json')

with open(spec_file) as f:
    specs = json.load(f)

In [ ]:
# Extract parameters for 1st-level analysis workflow
subject_list = specs['subject_list']
session_list = specs['session_list']
filters_spatial = specs['filters_spatial']
filters_temporal = specs['filters_temporal']
gm_mask_thr = specs['gm_mask_thr']
postfix = specs['multivariate_postfix']
clf_names = specs['clf_names']
sphere_radius = specs['sphere_radius']
sphere_steps = specs['sphere_steps']
n_chunks = specs['n_chunks']
tasks = specs['tasks']
n_perm = specs['n_perm']
atlasreader_names = specs['atlasreader_names']
atlasreader_prob_thresh = specs['atlasreader_prob_thresh']
n_proc = specs['n_parallel_jobs']

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject identifiers
subject_list 

In [ ]:
# List of session identifiers
session_list

In [ ]:
# List of spatial filters (smoothing) that were used during functional preprocessing
filters_spatial

In [ ]:
# List of temporal filters that were used during functional preprocessing
filters_temporal

In [ ]:
# Value to threshold gray matter probability template to create 2nd-level mask
gm_mask_thr

In [ ]:
# Specify a particular analysis postfix
postfix

In [ ]:
# List of classifier to use. Choose one or many of:
#   'LinearCSVMC', 'LinearNuSVMC', 'RbfCSVMC', 'RbfNuSVMC', 'SMLR', 'kNN', 'GNB'
clf_names

In [ ]:
# Searchlight sphere radius (in voxels), i.e. number of additional voxels
#    next to the center voxel. E.g sphere_radius = 3 means radius = 3.5*voxelsize
sphere_radius

In [ ]:
# Number of step size to define a sphere center, i.e. value of 5 means
#    that only every 5th voxel is used to perform a searchlight analysis
sphere_steps

In [ ]:
# Number of chunks to use for the N-Fold crossvalidation
#    (needs to divide number of labels without reminder)
n_chunks

In [ ]:
# Which classifications should be performed? (separated by task)
#    - Classification targets are a tuple of two tuples, indicating
#    - Target classification to train and target classification to test
tasks

In [ ]:
# Number of permutations to indicate group-analysis strategy:
#    n_perm = 0: group analysis is classical 2nd-level GLM analysis
#    n_perm > 0: group analysis is multivariate analysis according to Stelzer et al. (2013)
n_perm

In [ ]:
# Name of atlases to use for creation of output tables
atlasreader_names

In [ ]:
# Probability threshold to use for output tables
atlasreader_prob_thresh

In [ ]:
# Number of parallel jobs to run
n_proc

# Preparations

First things first, let's import important modules and specify relevant environment variables.

## Import modules

In [ ]:
import numpy as np
import os
from os.path import basename
from mvpa2.suite import *

## Specify environment variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

# Create multivariate output workflow
out_folder_name = 'analysis_multivariate'
if postfix:
    out_folder_name += '_%s' % postfix
out_path = opj(exp_dir, out_dir, out_folder_name)

# Create output folder
if not os.path.exists(out_path):
    os.makedirs(out_path)

# Function Definition

In this section, we will define all the functions that we need to run the searchlight analysis.

## Collect files

This function will return a list containing the beta-maps, a list containing the corresponding labels and a list that specifies which labels and contrasts belong to which chunk.

In [ ]:
def collect_files(subject_id, session_id, task_id, tFilter_id, sFilter_id, n_chunks):

    """This function collects the relevant input files, labels and chunk_idx"""
    
    # Get normalized 1st-level contrasts for specific parameters
    from bids.layout import BIDSLayout
    layout = BIDSLayout('/data/')

    # Collect contrast files
    search_parameters = {'subject': subject_id,
                         'task': task_id,
                         'return_type': 'file',
                         'type': 'norm'
                        }

    if session_id:
        search_parameters['session'] = session_id

    norm_files = layout.get(**search_parameters)

    # Collect label file
    search_parameters['type'] = 'labels'
    label_files = layout.get(**search_parameters)

    # Collect only requested files
    pass1 = '/data/derivatives/fmriflows/analysis_1stLevel'
    pass2 = '/multivariate'

    con_files = []
    labels = []

    for c in norm_files:
        if pass1 in c and pass2 in c:
            if tFilter_id in c and sFilter_id in c:
                con_files.append(c)

    for l in label_files:
        if pass1 in l and pass2 in l:
            if 'tFilter_' + tFilter_id in l and 'sFilter_' + sFilter_id in l:
                labels += list(np.loadtxt(l, dtype='S'))
                
    # Create chunks index
    chunks = [i for i in range(n_chunks)
              for j in range(int(len(con_files) / n_chunks))]
    
    # Make sure that that you have same number of contrasts and labels
    assert(len(con_files)==len(labels)==len(chunks))
    
    return sorted(con_files), labels, chunks

## Specify mask

This function will create and return a binary mask from the gray matter probability template to restrict the searchlight analysis.

In [ ]:
# Gray matter probability template to use for mask creation
mask_file = '/templates/mni_icbm152_nlin_asym_09c/1.0mm_tpm_gm.nii.gz'

In [ ]:
def create_mask(mask_file, con_file, gm_mask_thr, out_file):
    
    """Creates and returns a gray matter mask, that can be applied before
    the searchlight analysis."""

    from nilearn.image import resample_to_img, math_img, new_img_like
    from scipy.ndimage.morphology import binary_dilation

    # Resample mask image to contrast space and rescale to range of [0, 1]
    img_mask = resample_to_img(mask_file, con_file)
    mask = math_img('img/np.max(img) >= {}'.format(gm_mask_thr),
                    img=img_mask).get_data()

    # Apply binary dilation to image
    mask = binary_dilation(mask, iterations=2)
    img_mask = new_img_like(img_mask, mask, img_mask.affine)

    # Save image as a NIfTI file
    img_mask.to_filename(out_file)
    
    return out_file

## Searchlight results function

This function takes the results aggregated from the searchlight analysis and stores in every voxel of the mask the average value of each sphere that included this particular voxel. This function has therefore an effect of **smoothing the results**. Additionally, this function also allows to **fill up wholes** in the searchlight map if not every voxel was used as a center of a sphere.

In [ ]:
def fill_in_scattered_results(sl, dataset, roi_ids, results):

    """Function to aggregate results - This requires the searchlight
    conditional attribute 'roi_feature_ids' to be enabled"""
    
    resmap = None
    for resblock in results:
        for res in resblock:
            if resmap is None:
                
                # prepare the result container
                resmap = np.zeros((len(res), dataset.nfeatures),
                                  dtype=res.samples.dtype)
                observ_counter = np.zeros(dataset.nfeatures, dtype=int)
            
            # project the result onto all features -- love broadcasting!
            resmap[:, res.a.roi_feature_ids] += res.samples
            
            # increment observation counter for all relevant features
            observ_counter[res.a.roi_feature_ids] += 1
    
    # when all results have been added up average them according to the number
    # of observations
    observ_mask = observ_counter > 0
    resmap[:, observ_mask] /= observ_counter[observ_mask]
    result_ds = Dataset(resmap,
                        fa={'observations': observ_counter})
    
    if 'mapper' in dataset.a:
        import copy
        result_ds.a['mapper'] = copy.copy(dataset.a.mapper)
    
    return result_ds

## Classifier selection

This function return the classifier object defined by the classifier name `clf_name`.

In [ ]:
def get_classifier(clf_name):
    
    """Returns specified classifier object"""
    clfs = {
        'LinearCSVMC': LinearCSVMC(),
        'LinearNuSVMC': LinearNuSVMC(),
        'RbfCSVMC': RbfCSVMC(),
        'RbfNuSVMC': RbfNuSVMC(),
        'SMLR': SMLR(),
        'kNN': kNN(k=3),
        'GNB': GNB(),
    }

    return clfs[clf_name]

## PyMVPA dataset creation

This function creates the dataset object `ds` needed for the searchlight analysis. This is also where the data is normalized

In [ ]:
from mvpa2.base.hdf5 import h5save
from mvpa2.datasets.mri import fmri_dataset
from mvpa2.mappers.zscore import zscore

def create_dataset(subject_id, session_id, task_id, tFilter, sFilter, n_chunks, out_path, mask_file):
    
    """Create PyMVPA dataset for given input parameters and stores it in a HDF5 file"""
    
    # Create filter idx
    tFilter_id = '%s.%s' % (tFilter[0], tFilter[1])
    sFilter_id = '%s.%s' % (sFilter[0], sFilter[1])

    # Collect files, labels and chunks
    con_files, labels, chunks = collect_files(subject_id, session_id, task_id,
                                              tFilter_id, sFilter_id, n_chunks)
    
    # Create binary gray matter mask
    out_file = opj(out_path, 'gm_mask.nii.gz')
    mask_img = create_mask(mask_file, con_files[0], gm_mask_thr, out_file)
    
    # Create dataset
    ds = fmri_dataset(samples=con_files,
                      targets=labels,
                      chunks=chunks,
                      mask=mask_img)
    del ds.sa['time_coords']
    del ds.sa['time_indices']

    # Normalize dataset
    zscore(ds)
    
    # Save dataset in HDF5 format and return in
    ds_name = 'sub-%s_tFilter-%s_sFilter-%s_mask-%.03f.hdf5' % (
        subject_id, tFilter_id, sFilter_id, gm_mask_thr)
    if session_id:
        ds_name.replace('_tFilter', 'ses-%s_tFilter' % session_id)
    ds_path = opj(out_path, 'task-%s' % task_id, ds_name)
    h5save(ds_path, ds)
    
    return ds_path, mask_img, ds_name

## Create searchlight specific dataset

This function takes the PyMVPA dataset and prepares it for the searchlight analysis. In particular, it removes the target labels that are not needed for the classification and prepares the dataset for the cross-classification (training on one group to predict another) or the standard classification.

In [ ]:
def prepare_dataset(ds_path, train_labels, test_labels, cross_clf):

    """Prepares the dataset for the searchlight classification"""

    # Load dataset and extract labels and chunks
    ds = h5load(ds_path)
    labels = np.copy(ds.targets)
    chunks = np.copy(ds.chunks)
    
    # Select targets and rename chunks and labels if necessary
    if cross_clf:
        selecter = np.isin(labels, train_labels + test_labels)
        chunks = [int((l in test_labels) and ((l not in train_labels))) for l in labels[selecter]]
        selection = labels[selecter]
        for i, l in enumerate(test_labels):
            selection[np.argwhere(selection==test_labels[i])] = train_labels[i]
    else:
        selecter = np.isin(labels, train_labels)
        chunks = chunks[selecter]
        selection = labels[selecter]

    # Create searchlight analysis specifc dataset
    ds_sl = ds[selecter]
    ds_sl.sa.chunks = chunks
    ds_sl.sa.targets = selection

    return ds_sl

## Run searchlight analysis

This function runs the searchlight analysis with the given input parameters.

In [ ]:
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2.measures.base import CrossValidation
from mvpa2.measures.searchlight import sphere_searchlight
from mvpa2.misc.errorfx import mean_match_accuracy
from mvpa2.mappers.fx import mean_sample
from mvpa2.base import debug

def run_searchlight(ds_sl, clf_name, cross_clf, sphere_radius, sphere_steps, n_proc, verbose=False):

    """Run Searchlight Analysis and return searchlight output"""
    
    # Specify cross-validation scheme
    partitioner = NFoldPartitioner(cvtype=1)
    
    # Specify classifier
    clf = get_classifier(clf_name)
    
    # Create cross validation object
    if cross_clf:
        cv = CrossValidation(clf,
                             partitioner,
                             errorfx=mean_match_accuracy,
                             enable_ca=['stats'],
                             splitter=Splitter(attr='chunks',
                                               attr_values=(0, 1)))
    else:
        cv = CrossValidation(clf,
                             partitioner,
                             errorfx=mean_match_accuracy,
                             enable_ca=['stats'])

    # Create searchlight object
    sl = sphere_searchlight(cv,
                            radius=sphere_radius,
                            center_ids=range(0,
                                             ds_sl.shape[1],
                                             sphere_steps),
                            space='voxel_indices',
                            results_fx=fill_in_scattered_results,
                            postproc=mean_sample(),
                            enable_ca=['calling_time', 'roi_feature_ids'],
                            nproc=n_proc)
    
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    # Turn verbose on or off
    if verbose:
        debug.active = ["SLC"]
    else:
        debug.active = []
    
    # Run searchlight analysis
    sl_map = sl(ds_sl)
    
    return sl, sl_map

## Create informative logfile

This function writes relevant searchlight analysis information to a log file.

In [ ]:
from mvpa2.suite import time

def create_output(ds, sl, sl_map, subject_id, clf_name, targets, sphere_radius,
                  sphere_steps, ds_name, clf_type, result_path, n_proc):
    
    """Save important model information in a text file."""

    # Extract important information
    wall_time = time.strftime('%H:%M:%S', time.gmtime(round(sl.ca.calling_time)))

    # Accuracy information
    accuracies = sl_map.S[0]
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    chance_level = 1.0 / len(np.unique(ds.sa.targets))

    # Helper functions
    def threshold_above_average(x):
        return chance_level + x * std_accuracy

    def spheres_above_average(x):
        return np.sum(accuracies >= threshold_above_average(x))

    def percent_above_average(x):
        return np.mean(accuracies >= threshold_above_average(x)) * 100
    
    # Output text
    txt = ['Subject          : {0}'.format(subject_id),
           'Classifier       : {0}'.format(clf_name),
           'Classes          : {0}'.format(targets),
           'Sphere Radius    : {0}'.format(sphere_radius),
           'N-th Element     : {0}'.format(sphere_steps),
           'Wall Time        : {0}'.format(wall_time),
           'Samples          : {0}'.format(ds.S.shape[0]),
           'Features         : {0}'.format(ds.S.shape[1]),
           'Volume Dimension : {0}'.format(str(ds.a.voxel_dim)),
           'Voxel  Dimension : {0}'.format(str(ds.a.voxel_eldim)),
           'CPU              : {0}'.format(n_proc),
           
           '\nChance Level     : {0:>5}'.format(round(chance_level, 5)*100),
           'Accuracy (mean)  : {0:>5}'.format(round(mean_accuracy, 5)*100),
           'Accuracy (std)   : {0:>5}'.format(round(std_accuracy, 5)*100),
           
           'above 2STD in %  : {0:>7}%'.format(round(percent_above_average(2), 3)),
           'above 3STD in %  : {0:>7}%'.format(round(percent_above_average(3), 3)),
           'above 2STD in v  : {0:>7}'.format(spheres_above_average(2)),
           'above 3STD in v  : {0:>7}'.format(spheres_above_average(3)),
           
           '\nDataset Summary:',
           '****************',
           '%s' % ds.summary(),
           '%s' % ds.sa,
           '\n%s' % ds.summary]

    # Write information to text file
    results_file = opj(result_path, ds_name.replace('.hdf5', '_%s.rst' % clf_type))
    with open(results_file, 'w') as f:
        f.writelines('\n'.join(txt))
        
    # Return relevant information
    results = [wall_time, chance_level, mean_accuracy, std_accuracy,
               spheres_above_average(2), spheres_above_average(3)]
    return results_file, results

## Save searchlight result in NIfTI and show it in glassbrain plot

This function stores the searchlight results in a NIfTI file and shows the output on a glassbrain plot, thresholded at a given value.

In [ ]:
from nilearn.plotting import plot_glass_brain
from nilearn.image import new_img_like

def plot_glassbrain(ds, sl_map, mask_img, results_file, threshold):

    # Put searchlight output back into NIfTI space
    sl_data = ds.mapper.reverse(sl_map.S)[0, ...]
    sl_img = new_img_like(mask_img, sl_data)
    sl_filename = results_file.replace('.rst', '.nii.gz')
    sl_img.to_filename(opj(result_path, sl_filename))

    # Plotting the searchlight results on the glass brain
    title_txt = '{} - threshold = {}%'.format(clf_type, round(threshold * 100, 2))
    plot_glass_brain(sl_img, black_bg=True, colorbar=True, title=title_txt,
                     display_mode='lyrz', threshold=threshold, cmap='magma',
                     output_file=results_file.replace('.rst', '.png'))

# Perform Searchlight Analysis

Now that all important functions are specified, we can perform the searchlight analysis. As with a classical design, searchlight analysis apply a 1st-level and 2nd-level analysis. The **correct approach** to perform a 2nd-level analysis on searchlight results, i.e. accuracy maps, involves permutation (see [Stelzer et al. (2013)](https://www.sciencedirect.com/science/article/pii/S1053811912009810)). This notebook nonetheless, applies also a classical 2nd-level GLM, that can be used as an indication of results, but shouldn't be used for publications!

In the rest of the notebook the following steps are conducted:

1. **1st-level searchlight analysis** (with original labels) to acquire accuracy maps
1. **classical 2nd-level GLM analysis**, based on those accuracy maps
1. **1st-level searchlight analysis** with **permutated labels** (performed N-times)
1. **2nd-level analysis** according **[Stelzer et al. (2013)](https://www.sciencedirect.com/science/article/pii/S1053811912009810)**

## 1st-level searchlight analysis (with original labels)

In [ ]:
# Specify order of parameter to iterate over
if not session_list:
    session_list = ['']
    
iteration_list = [[s, t, tf, sf, sess]
                  for sess in session_list
                  for tf in filters_temporal
                  for sf in filters_spatial
                  for t in tasks.keys()
                  for s in subject_list]
iteration_list

In [ ]:
# Iterate over all parameters and run the searchlight analysis
for subject_id, task_id, tFilter, sFilter, session_id in iteration_list:

    # Create multivariate dataset
    ds_path, mask_img, ds_name = create_dataset(
        subject_id, session_id, task_id, tFilter, sFilter,
        n_chunks, out_path, mask_file)

    # Go through all targets
    for targets in tasks[task_id]:

        # Extract training and testing labels
        train_labels = targets[0]
        test_labels = targets[1]

        # Specify results folder
        result_path = opj(out_path, 'task-%s' % task_id, 'train-%s_test-%s' % (
            '_'.join([str(s) for s in targets[0]]),
            '_'.join([str(s) for s in targets[1]])))
        if not os.path.exists(result_path):
            os.makedirs(result_path)

        # Are the training and target labels the same?
        cross_clf = targets[0] != targets[1]

        # Prepare PyMVPA dataset for searchlight analysis
        ds_sl = prepare_dataset(ds_path, train_labels, test_labels, cross_clf)

        # Go through specified classifiers
        for clf_name in clf_names:

            # Perform searchlight analysis
            sl, sl_map = run_searchlight(
                ds_sl, clf_name, cross_clf, sphere_radius, sphere_steps, n_proc,
                verbose=False)

            # Create visual outputs and report file
            clf_type = 'clf-%s_radius-%0d_steps-%03d' % (
                clf_name, sphere_radius, sphere_steps)
            results_file, results = create_output(
                ds_sl, sl, sl_map, subject_id, clf_name, targets,
                sphere_radius, sphere_steps, ds_name, clf_type,
                result_path, n_proc)
            wall_time, chance_level, mean_accuracy, std_accuracy, v2STD, v3STD = results
            threshold = mean_accuracy + 2 * std_accuracy
            plot_glassbrain(ds_sl, sl_map, mask_img, results_file, threshold)

            # Print log stream to terminal
            out_stream = '{} - Targets: {}'.format(
                ds_name.replace('.hdf5', ''), '_'.join([str(t) for t in targets]))
            out_stream += '\n  {}  Radius: {:>5}   Steps: {:>4}   CLF: {}'.format(
                wall_time, sphere_radius, sphere_steps, clf_name)
            out_stream += '\n            Chance: {:>5}%  Mean: {:>5}%  STD: {}%  +2STDv={} +3STDv={}'.format(
                round(100*chance_level, 1), round(100*mean_accuracy, 1), round(100*std_accuracy, 1), v2STD, v3STD)
            border_txt = '#' * 40
            print('\n'.join([border_txt, out_stream, border_txt]))

## Classical 2nd-level GLM analysis

The application of a classical GLM approach to perform a 2nd-level analysis of searchlight results is not recommended. The correct way to perform a group analysis on searchlight accuracy maps is by applying permutation testing, as for example proposed by [Stelzer et al. (2013)](https://www.sciencedirect.com/science/article/pii/S1053811912009810)).

Having said all this, the following code performs a classical 2nd-level GLM analysis and tests the acquired accuracy maps against chance level.

In [ ]:
# Specify order of parameter to iterate over
if not session_list:
    session_list = ['']
    
iteration_list = [[t, tf, sf, sess]
                  for sess in session_list
                  for tf in filters_temporal
                  for sf in filters_spatial
                  for t in tasks.keys()]
iteration_list

In [ ]:
import pandas as pd
from glob import glob
from nilearn.image import math_img
from nilearn.plotting import plot_glass_brain
from nistats.second_level_model import SecondLevelModel
from nistats.thresholding import map_threshold

# Iterate over all parameters and run the searchlight analysis
for task_id, tFilter, sFilter, session_id in iteration_list:

    # Create filter idx
    tFilter_id = '%s.%s' % (tFilter[0], tFilter[1])
    sFilter_id = '%s.%s' % (sFilter[0], sFilter[1])

    # Go through all targets
    for targets in tasks[task_id]:
        
        # Specify classification folder
        clf_folder = 'train-%s_test-%s' % (
            '_'.join([str(s) for s in targets[0]]),
            '_'.join([str(s) for s in targets[1]]))

        # Go through specified classifiers
        for clf_name in clf_names:
            
            # Collect all the files
            file_idx = 'sub-*_tFilter-%s_sFilter-%s_mask-%.03f' % (tFilter_id, sFilter_id, gm_mask_thr)
            file_idx += '_clf-%s_radius-%0d_steps-%03d.nii.gz' % (clf_name, sphere_radius, sphere_steps)
            sl_res = sorted(glob(opj(out_path, 'task-%s' % task_id, clf_folder, file_idx)))
            
            # Path to output folder
            result_path = opj(out_path, 'task-%s' % task_id, 'group_glm', clf_folder)
            if not os.path.exists(result_path):
                os.makedirs(result_path)
                
            # Center searchlight accuracy maps around chance level
            chance_level = 1.0 / len(targets[0])
            sl_imgs = [math_img('img - %s' % chance_level, img=sl) for sl in sl_res]
            
            # Specify 2nd-level GLM design matrix
            design_matrix = pd.DataFrame([1] * len(sl_res), columns=['intercept'])

            # Estimate 2nd-level Model
            second_level_model = SecondLevelModel()
            second_level_model = second_level_model.fit(
                sl_imgs, design_matrix=design_matrix)

            # Compute z-score
            z_map = second_level_model.compute_contrast(output_type='z_score')
            out_filename = opj(result_path, file_idx.replace('sub-*', 'z-map'))
            z_map.to_filename(out_filename)

            # Threshold output with different approaches and save figure
            for l, h in [(.001, 'fpr'), (.05, 'fdr'), (.05, 'bonferroni')]:
            
                thr_name = out_filename.replace('z-map', 'thr_%s_%.03f' % (h, l))
                thr_map, thr = map_threshold(z_map, level=l, height_control=h)
                thr_map.to_filename(thr_name)
                
                plot_glass_brain(
                    z_map, threshold=thr, colorbar=True, black_bg=True, plot_abs=False,
                    display_mode='lyrz',  output_file=thr_name.replace('.nii.gz', '.png'),
                    title='%s: %s - Threshold = %.03f' % (h, l, thr))
            
            log_txt = file_idx.replace('sub-*_', '').replace('.nii.gz', '').split('_')
            print('{:<20} {:<16} {:<12} {:<18} {:<10} {:<12} finished.'.format(*log_txt))

## 1st-level searchlight analysis (with label permutation)

Depending on the number of permutations per subject, this step will take a long time to compute. It is recommended to take the following code and run it on a cluster server, where you can profit from real parallelization.

In [ ]:
# Specify order of parameter to iterate over
if not session_list:
    session_list = ['']
    
iteration_list = [[s, t, tf, sf, sess]
                  for sess in session_list
                  for tf in filters_temporal
                  for sf in filters_spatial
                  for t in tasks.keys()
                  for s in subject_list]
iteration_list

In [ ]:
# Iterate over all parameters and run the searchlight analysis
for subject_id, task_id, tFilter, sFilter, session_id in iteration_list:

    # Create multivariate dataset
    ds_path, mask_img, ds_name = create_dataset(
        subject_id, session_id, task_id, tFilter, sFilter,
        n_chunks, out_path, mask_file)

    # Create filter idx
    tFilter_id = '%s.%s' % (tFilter[0], tFilter[1])
    sFilter_id = '%s.%s' % (sFilter[0], sFilter[1])

    # Go through all targets
    for targets in tasks[task_id]:

        # Extract training and testing labels
        train_labels = targets[0]
        test_labels = targets[1]

        # Specify results folder
        result_path = opj(out_path, 'task-%s' % task_id, 'train-%s_test-%s' % (
            '_'.join([str(s) for s in targets[0]]),
            '_'.join([str(s) for s in targets[1]])), 'permutations')
        if not os.path.exists(result_path):
            os.makedirs(result_path)

        # Are the training and target labels the same?
        cross_clf = targets[0] != targets[1]

        # Prepare PyMVPA dataset for searchlight analysis
        ds_sl = prepare_dataset(ds_path, train_labels, test_labels, cross_clf)

        # Go through specified classifiers
        for clf_name in clf_names:

            # Go through all permutations, but verify that none occures double
            perm = AttributePermutator('targets')

            used_combinations = []
            used_combinations.append(''.join(ds_sl.targets))

            p = 0
            while p < n_perm - 1:

                # Permutate dataset
                ds_perm = perm(ds_sl)

                # Catch searchlight analysis that break
                try:
                
                    # Verify that permutation is unique
                    counter = 0
                    while (''.join(ds_perm.targets) in used_combinations) and counter < n_perm:
                        ds_perm = perm(ds_sl)
                        counter += 1

                    # Perform searchlight if permutation limit is not reached yet
                    if counter != n_perm:

                        used_combinations.append(''.join(ds_perm.targets))

                        sl, sl_map = run_searchlight(
                            ds_perm, clf_name, cross_clf, sphere_radius, sphere_steps, n_proc,
                            verbose=False)

                        # Save result in a numpy object
                        out_name = 'sub-%s_tFilter-%s_sFilter-%s_mask-%.03f' % (
                            subject_id, tFilter_id, sFilter_id, gm_mask_thr)
                        out_name += '_clf-%s_radius-%0d_steps-%03d' % (
                            clf_name, sphere_radius, sphere_steps)
                        out_name += '_nperm-%03d.npy' % (p + 1)

                        if session_id:
                            out_name.replace('_tFilter', 'ses-%s_tFilter' % session_id)

                        np.save(opj(result_path, out_name), sl_map.samples)

                        # Print log stream to terminal
                        wall_time = time.strftime('%H:%M:%S', time.gmtime(round(sl.ca.calling_time)))
                        print('{} {}'.format(wall_time, basename(out_name)[:-4]))

                    p += 1
                
                except FailedToTrainError as e:
                    print 'Searchlight was restarted because of error: %s' % e

## 2nd-level analysis according to [Stelzer et al. (2013)](https://www.sciencedirect.com/science/article/pii/S1053811912009810)

This next step is also time consuming, but luckily is much lower than the previous permutation process. It should be finished within a few hours.

First, we need to collect all the data:

In [ ]:
# Specify order of parameter to iterate over
if not session_list:
    session_list = ['']
    
iteration_list = [[t, tf, sf, sess]
                  for sess in session_list
                  for tf in filters_temporal
                  for sf in filters_spatial
                  for t in tasks.keys()]
iteration_list

In [ ]:
from glob import glob

# Iterate over all parameters and run the searchlight analysis
for task_id, tFilter, sFilter, session_id in iteration_list:

    # Create filter idx
    tFilter_id = '%s.%s' % (tFilter[0], tFilter[1])
    sFilter_id = '%s.%s' % (sFilter[0], sFilter[1])

    # Go through all targets
    for targets in tasks[task_id]:
        
        # Specify classification folder
        clf_folder = 'train-%s_test-%s' % (
            '_'.join([str(s) for s in targets[0]]),
            '_'.join([str(s) for s in targets[1]]))

        # Go through specified classifiers
        for clf_name in clf_names:

            # Path to output folder
            result_path = opj(out_path, 'task-%s' % task_id, 'group_stelzer', clf_folder)
            if not os.path.exists(result_path):
                os.makedirs(result_path)
                
            # Collect all files
            file_idx = 'sub-*_tFilter-%s_sFilter-%s_mask-%.03f' % (
                tFilter_id, sFilter_id, gm_mask_thr)
            file_idx += '_clf-%s_radius-%0d_steps-%03d.nii.gz' % (
                clf_name, sphere_radius, sphere_steps)
            path_orig = sorted(glob(opj(
                out_path, 'task-%s' % task_id, clf_folder, file_idx)))
            path_perm = sorted(glob(opj(
                out_path, 'task-%s' % task_id, clf_folder, 'permutations',
                file_idx.replace('.nii.gz', '_nperm-*.npy'))))
            
            # Extract data from all fils
            mask_id = 'sub-%s_tFilter-%s_sFilter-%s_mask-%.03f.hdf5' % (
                subject_list[0], tFilter_id, sFilter_id, gm_mask_thr)
            if session_id:
                ds_name.replace('_tFilter', 'ses-%s_tFilter' % session_id)
            mask_mapper = h5load(opj(out_path, 'task-%s' % task_id, mask_id))
            acc_orig = np.vstack([mask_mapper.a.mapper[:2].forward1(
                nb.load(orig).get_data()) for orig in path_orig])
            acc_perm = np.vstack([np.load(perm).ravel() for perm in path_perm])
            
            # Create PyMVPA datasets for group analysis
            orig_ds = Dataset(acc_orig,
                              sa=dict(subj=subject_list),
                              fa=mask_mapper.fa, a=mask_mapper.a)
            perm_ds = Dataset(np.vstack((acc_orig, acc_perm)),
                              sa=dict(subj=np.hstack(
                                  (subject_list,
                                   np.repeat(subject_list, n_perm - 1)))),
                              fa=mask_mapper.fa, a=mask_mapper.a)


In [ ]:
from mvpa2.datasets.mri import map2nifti
from mvpa2.algorithms.group_clusterthr import GroupClusterThreshold

# Debugger on or off
if __debug__:
    from mvpa2.base import debug
    debug.active = ['GCTHR']

In [ ]:
threshold = 0.001
thr = GroupClusterThreshold(n_bootstrap=1000,
                            feature_thresh_prob=threshold,
                            chunk_attr='subj',
                            fwe_rate=0.05,
                            multicomp_correction='fdr_bh',
                            n_proc=n_proc)
# bonferroni, sidak, holm-sidak, holm, simes-hochberg, hommel, fdr_bh, fdr_by, None

In [ ]:
%%time
thr.train(perm_ds)

In [ ]:
res = thr(orig_ds)
h5save(opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
           'grpavg_stats_%s_thr%s.hdf5' % (identifier, threshold)),
       res, compression=9)

In [ ]:
# Vector with average accuracy
nb.save(map2nifti(res, res.samples),
        opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
            'grpavg_avg_acc_%s_thr%s.nii.gz' % (identifier, threshold)))

In [ ]:
# Vector with feature-wise cluster-forming thresholds.
nb.save(map2nifti(res, res.fa.featurewise_thresh),
        opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
            'grpavg_featurewise_thresh_%s_thr%s.nii.gz' % (identifier,
                                                           threshold)))

In [ ]:
# Vector with labels for clusters after thresholding. Cluster values are sorted
# by cluster size. The largest cluster is always labeled with ``1``.
nb.save(map2nifti(res, res.fa.clusters_featurewise_thresh),
        opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
            'grpavg_clusters_featurewise_thresh_%s_thr%s.nii.gz' % (identifier,
                                                                    threshold)))

In [ ]:
# Same Vector as above but with accuracies
nb.save(map2nifti(res, np.array(res.fa.clusters_featurewise_thresh != 0).astype('int') * res.samples),
        opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
            'grpavg_clusters_featurewise_thresh_%s_thr%s_acc.nii.gz' % (identifier,
                                                                        threshold)))

In [ ]:
# Vector with labels for super-threshold clusters after FWE correction. The
# attribute is derived from ``fa.clusters_featurewise_thresh`` by removing all
# clusters that do not pass the threshold when controlling for FWE.
if hasattr(res.fa, 'clusters_fwe_thresh'):
    nb.save(map2nifti(res, res.fa.clusters_fwe_thresh),
            opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
                'grpavg_clusters_fwe_thresh_%s_thr%s.nii.gz' % (identifier,
                                                                threshold)))

In [ ]:
# Same Vector as above but with accuracies
nb.save(map2nifti(res, np.array(res.fa.clusters_fwe_thresh != 0).astype('int') * res.samples),
        opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
            'grpavg_clusters_fwe_thresh_%s_thr%s_acc.nii.gz' % (identifier,
                                                                threshold)))

In [ ]:
# Write out the record array with information on all detected clusters. The array
# contains the fields size (number of features comprising the cluster), and
# prob_raw (probability of observing the cluster of a this size or larger under
# the NULL hypothesis). If correction for multiple comparisons is enabled an
# additional field prob_corrected (probability after correction) is added.
# The array also contains the fields ``max`` (feature coordinate of the maximum
# score within the cluster, and ``center_of_mass`` (coordinate of the center of
# mass; weighted by the feature values within the cluster.
with open(opj(experiment_dir, 'medusa_new', 'results_group_%s' % identifier,
              'clusterstats_%s_thr%s.csv' % (identifier, threshold)),
          'w') as sFile:

    header = [e[0] for e in res.a.clusterstats.dtype.descr] + [e[0]
                                                               for e in res.a.clusterlocations.dtype.descr]

    content = [','.join([str(e) for e in c] +
                        ['(%s)' % ' '.join([str(l[0]) for l in res.a.clusterlocations[i]]),
                         '(%s)' % ' '.join([str(l[1]) for l in res.a.clusterlocations[i]])])
               for i, c in enumerate(res.a.clusterstats)]

    sFile.write(','.join(header) + '\n')
    sFile.write('\n'.join(content))